In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from xgboost import XGBClassifier

from lightgbm import LGBMClassifier

from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('../inputs/train.csv')

In [4]:
# remove unnecesary columns
single_uniq_value_cols = ['cat_var_31', 'cat_var_35', 'cat_var_36', 'cat_var_37', 'cat_var_38', 'cat_var_40', 'cat_var_42']
single_majority_value_cols = ['cat_var_23', 'cat_var_24', 'cat_var_25', 'cat_var_26',\
                              'cat_var_27', 'cat_var_28', 'cat_var_29', 'cat_var_30',\
                              'cat_var_32', 'cat_var_33', 'cat_var_34', 'cat_var_39',\
                              'cat_var_41']

In [5]:
df.drop(single_uniq_value_cols, axis = 1, inplace = True)
df.drop(single_majority_value_cols, axis = 1, inplace = True)

In [6]:
df.fillna('NA', inplace = True)

In [7]:
print (df.dtypes)

transaction_id     object
num_var_1         float64
num_var_2         float64
num_var_3         float64
num_var_4         float64
num_var_5         float64
num_var_6         float64
num_var_7         float64
cat_var_1          object
cat_var_2          object
cat_var_3          object
cat_var_4          object
cat_var_5          object
cat_var_6          object
cat_var_7          object
cat_var_8          object
cat_var_9          object
cat_var_10         object
cat_var_11         object
cat_var_12         object
cat_var_13         object
cat_var_14         object
cat_var_15         object
cat_var_16         object
cat_var_17         object
cat_var_18         object
cat_var_19          int64
cat_var_20          int64
cat_var_21          int64
cat_var_22          int64
target              int64
dtype: object


In [14]:
input_cols_lgb_encode = df.dtypes[~df.dtypes.isin([np.int64, np.float64])].index.tolist()
print (input_cols_lgb_encode)
input_cols_lgb_encode.remove('transaction_id')
input_cols_lgb_numeric = df.dtypes[df.dtypes.isin([np.int64, np.float64])].index.tolist()
print (input_cols_lgb_numeric)
input_cols_lgb_numeric.remove('target')
target_cols_lgb = ['target']

['transaction_id', 'cat_var_1', 'cat_var_2', 'cat_var_3', 'cat_var_4', 'cat_var_5', 'cat_var_6', 'cat_var_7', 'cat_var_8', 'cat_var_9', 'cat_var_10', 'cat_var_11', 'cat_var_12', 'cat_var_13', 'cat_var_14', 'cat_var_15', 'cat_var_16', 'cat_var_17', 'cat_var_18']
['num_var_1', 'num_var_2', 'num_var_3', 'num_var_4', 'num_var_5', 'num_var_6', 'num_var_7', 'cat_var_19', 'cat_var_20', 'cat_var_21', 'cat_var_22', 'target']


In [15]:
X_lgb = df[input_cols_lgb_encode + input_cols_lgb_numeric].as_matrix()
label_enc = [LabelEncoder() for i in range(len(input_cols_lgb_encode))]

for i in range(len(label_enc)):
    print (input_cols_lgb_encode[i])
    label_enc[i].fit(X_lgb[:, i])
    X_lgb[:, i] = label_enc[i].transform(X_lgb[:, i])

y_lgb = df['target']

cat_var_1
cat_var_2
cat_var_3
cat_var_4
cat_var_5
cat_var_6
cat_var_7
cat_var_8
cat_var_9
cat_var_10
cat_var_11
cat_var_12
cat_var_13
cat_var_14
cat_var_15
cat_var_16
cat_var_17
cat_var_18


In [16]:
# do stratified splitting as training time is too large otherwise
X_lgb_train, X_lgb_test, y_lgb_train, y_lgb_test = train_test_split(X_lgb, y_lgb, test_size = 0.4, \
                                                                    stratify = y_lgb, random_state = 42)

In [19]:
# get the optimum no of trees for learning rate of 0.3
parameters = {'n_estimators' : [10, 50, 100]}
lgb_clf = LGBMClassifier(random_state=42, n_jobs = -1,learning_rate = 0.1)
clf = GridSearchCV(lgb_clf, parameters, cv = 4, scoring = 'roc_auc')
clf.fit(X_lgb_train, y_lgb_train)
print (clf.best_params_)

LightGBMError: b'Unknown parameter: random_state'